In [ ]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

# install findspark using pip
!pip install -q findspark

# install pyspark
!pip3 install pyspark==3.2.0

# install graphframes
!pip3 install graphframes

In [ ]:
!cp -v /content/graphframes-0.8.2-spark3.2-s_2.12.jar $SPARK_HOME/jars/

'/content/graphframes-0.8.2-spark3.2-s_2.12.jar' -> '/content/spark-3.2.0-bin-hadoop3.2/jars/graphframes-0.8.2-spark3.2-s_2.12.jar'


In [ ]:
#import the packages
from pyspark import *
from pyspark.sql import *
from graphframes import *
import findspark
import pandas as pd

findspark.init()

# Start a Spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
import psycopg2
import pandas as pd

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
#establishing the connection
conn = psycopg2.connect(
   database="postgres", user='cpdbstudent', password='DataSci4AI', host='codd01.research.northwestern.edu', port= '5432'
)

In [ ]:
cursor = conn.cursor()

In [ ]:
edges_query = '''
DROP TABLE IF EXISTS ids_distinct;
create temp table ids_distinct as(
select distinct a.id
   	from data_officer a
join (
SELECT rank, sum(total_allegations) as Total_allegations
       from (select a.id, a.first_name, a.last_name, a.rank, count(b.officer_id) as total_allegations
   	from data_officer a
join data_officerallegation b
on a.id = b.officer_id
group by a.id
order by count(b.officer_id) desc)A group by rank order by total_allegations desc limit 10) b
on a.rank = b.rank);

select src, dst, relationship from(
SELECT da1.officer_id src, da2.officer_id dst, COUNT(DISTINCT da1.allegation_id) relationship
FROM data_officerallegation da1
JOIN data_officerallegation da2 ON da1.allegation_id = da2.allegation_id AND da1.officer_id < da2.officer_id
GROUP BY da1.officer_id, da2.officer_id ORDER BY count(*) DESC)popln_sample
JOIN ids_distinct ON ids_distinct.id = popln_sample.src;

'''

In [ ]:
nodes_query = '''
DROP TABLE IF EXISTS ids_distinct;
create temp table ids_distinct as(
select distinct a.id, a.rank
   	from data_officer a
join (
SELECT rank, sum(total_allegations) as Total_allegations
       from (select a.id, a.first_name, a.last_name, a.rank, count(b.officer_id) as total_allegations
   	from data_officer a
join data_officerallegation b
on a.id = b.officer_id
group by a.id
order by count(b.officer_id) desc)A group by rank order by total_allegations desc limit 10) b
on a.rank = b.rank);

DROP TABLE IF EXISTS edges_q1;
create temp table edges_q1 as(
select src, dst, relationship from(
SELECT da1.officer_id src, da2.officer_id dst, COUNT(DISTINCT da1.allegation_id) relationship
FROM data_officerallegation da1
JOIN data_officerallegation da2 ON da1.allegation_id = da2.allegation_id AND da1.officer_id < da2.officer_id
GROUP BY da1.officer_id, da2.officer_id ORDER BY count(*) DESC)popln_sample
JOIN ids_distinct ON ids_distinct.id = popln_sample.src);

SELECT id, rank, first_name || ' ' || last_name officer_name, allegation_count FROM data_officer
where id in (select distinct src from edges_q1);
'''

In [ ]:
cursor.execute(edges_query)
edges = cursor.fetchall()
print("shape is: " + str(len(edges))) # 17465

df_edges = pd.DataFrame(edges)
colnames = [desc[0] for desc in cursor.description]
df_edges.columns = colnames

shape is: 221740


In [ ]:
cursor.execute(nodes_query)
nodes = cursor.fetchall()
print("shape is: " + str(len(nodes))) # 17465

df_nodes = pd.DataFrame(nodes)
colnames = [desc[0] for desc in cursor.description]
df_nodes.columns = colnames

shape is: 18192


In [ ]:
df_nodes

,id,rank,officer_name,allegation_count
0,24416,Police Officer,Michael Rodriguez,12
1,4790,Police Officer,Pablo Claudio,2
2,26745,Sergeant of Police,Richard Smith,3
3,3936,Police Officer,Ronald Carlson,5
4,18803,Police Officer,Melvin Mendez,5
...,...,...,...,...
18187,536,Police Officer,Alphonse Amezaga,2
18188,19812,Sergeant of Police,Patrick Moyer,40
18189,4035,Police Officer,Michael Carroll,2
18190,790,Police Officer,James Arceo,68


In [ ]:
edges_ = spark.createDataFrame(df_edges)

In [ ]:
nodes = spark.createDataFrame(df_nodes)

In [ ]:
cpdb2 = GraphFrame(nodes, edges_)

In [ ]:
tc_cpdb = cpdb2.triangleCount()
tc_cpdb.orderBy('count', ascending=False).show()
tc_cpdb.select("id", "count").show()

+-----+-----+--------------------+------------------+----------------+
|count|   id|                rank|      officer_name|allegation_count|
+-----+-----+--------------------+------------------+----------------+
|29636| 3033|  Sergeant of Police|    Raimondo Brown|              17|
|29633| 6315|      Police Officer|     Terence Davis|              38|
|29597| 3744|      Police Officer|    Derek Campbell|               8|
|25678|18042|      Police Officer|     Donald Mc Coy|              22|
|25646|  441|      Police Officer|   Fernando Alonzo|              16|
|22131|21530|      Police Officer|Michael Overstreet|              56|
|21846|27349|      Police Officer|   Charles Stanton|              11|
|21836| 5180|  Sergeant of Police|    Stephen Conner|               9|
|21824| 5667|      Police Officer|     Jerry Crawley|              30|
|21814|16747|      Police Officer|     Evetta Lundin|               7|
|21812| 8844|      Police Officer|      Thomas Flynn|              19|
|21809

In [ ]:
pr_cpdb = cpdb2.pageRank(resetProbability=0.15, tol=0.01)
#look at the pagerank score for every vertex
pr_cpdb.vertices.orderBy('pagerank', ascending=False).show()
pr_cpdb.edges.show()

+-----+--------------------+------------------+----------------+------------------+
|   id|                rank|      officer_name|allegation_count|          pagerank|
+-----+--------------------+------------------+----------------+------------------+
|32350|      Police Officer|    Robert Spiegel|              20| 59.94255110579391|
|32351|      Police Officer|  Boonserm Srisuth|              25| 58.47588273428055|
|32419|      Police Officer|         Eric Wier|              18| 55.54706144219321|
|32284|      Police Officer|         Mark Reno|              76| 55.05553771973534|
|32384|      Police Officer|     Edwin Utreras|              47| 52.46541790094154|
|32337|Police Officer as...|       Louis Silva|              21| 51.72556067035757|
|32413|      Police Officer| Carl Weatherspoon|              69|  50.3834356160415|
|32289|      Police Officer|       John Rivera|              44| 49.31297984845296|
|32074|Police Officer as...|    Ronald Jenkins|              46| 47.94275095